In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from utils.data import CatsAndDogs
from torch.utils.data import DataLoader
from torch import nn, optim
from torch.nn import functional as F
from models.classifier import ClassifierForCatsAndDogs

In [2]:
BATCH_SIZE = 16
ETA = 1e-4
EPOCHS = 20

In [3]:
def compute_accuracy(ps, labels):
    equality = (torch.round(ps) == labels).float()
    acc = equality.mean()
    return acc

In [4]:
def train_step(context, x, y):
    model = context["model"]
    optimizer = context["optimizer"]
    
    optimizer.zero_grad()
    
    ps = model(x)
    loss = F.binary_cross_entropy(ps.squeeze(), y) # nn.BCELoss()(ps, y) 와 같음
    
    loss.backward()
    optimizer.step()
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [5]:
def eval_step(context, x, y):
    model = context["model"]
    
    ps = model(x)
    loss = F.binary_cross_entropy(ps.squeeze(), y) # nn.BCELoss()(ps, y) 와 같음
    
    acc = compute_accuracy(ps, y)
    
    return loss.item(), acc.item()

In [6]:
def train():
    trainset = CatsAndDogs(mode="train")
    train_loader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
    
    testset = CatsAndDogs(mode="test")
    test_loader = DataLoader(testset, batch_size=BATCH_SIZE)
    
    model = ClassifierForCatsAndDogs().cuda()
    optimizer = optim.Adam(model.parameters())
    
    context = {
        "model": model,
        "optimizer": optimizer
    }
    
    for e in range(EPOCHS):
        train_loss = 0.0
        train_acc = 0.0
        test_loss = 0.0
        test_acc = 0.0
        
        model.train()
        
        for x, y in train_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            loss, acc = train_step(context, x, y)
            train_loss += loss
            train_acc += acc
            
        model.eval()
            
        for x, y in test_loader:
            x = x.cuda()
            y = y.float().cuda()
            
            with torch.no_grad():
                loss, acc = eval_step(context, x, y)
                test_loss += loss
                test_acc += acc
                
        train_loss /= len(train_loader)
        train_acc /= len(train_loader)
        test_loss /= len(test_loader)
        test_acc /= len(test_loader)
        
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train loss: {train_loss:.8f}, train acc: {train_acc:.4f}")
        print(f"Test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")

In [7]:
train()

Epochs 1/20
Train loss: 0.69559437, train acc: 0.4815
Test loss: 0.69927099, test acc: 0.4911
Epochs 2/20
Train loss: 0.67358583, train acc: 0.6296
Test loss: 0.70240709, test acc: 0.3973
Epochs 3/20
Train loss: 0.65808427, train acc: 0.6412
Test loss: 0.73590100, test acc: 0.4688
Epochs 4/20
Train loss: 0.62820077, train acc: 0.7153
Test loss: 0.70728040, test acc: 0.5357
Epochs 5/20
Train loss: 0.60059261, train acc: 0.7361
Test loss: 0.75647756, test acc: 0.4866
Epochs 6/20
Train loss: 0.55906087, train acc: 0.7685
Test loss: 0.72725889, test acc: 0.5513
Epochs 7/20
Train loss: 0.52110539, train acc: 0.8264
Test loss: 0.75850467, test acc: 0.5536
Epochs 8/20
Train loss: 0.47859054, train acc: 0.8472
Test loss: 0.74806240, test acc: 0.5179
Epochs 9/20
Train loss: 0.44657960, train acc: 0.8218
Test loss: 0.75562802, test acc: 0.5112
Epochs 10/20
Train loss: 0.41675978, train acc: 0.8333
Test loss: 0.75821543, test acc: 0.4866
Epochs 11/20
Train loss: 0.36084943, train acc: 0.8819
Test